In [2]:
!pip install music21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 40.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00


In [4]:
import os
import glob
from tqdm import tqdm
from music21 import converter, instrument, note, chord, stream, pitch
import numpy as np
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
from keras.metrics import Mean
from keras.layers import Input, LSTM, Dense, Lambda, Layer
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import KLDivergence, CategoricalCrossentropy

### Load and preprocess dataset

In [4]:
# Corrupt files / malformed structured for music21 library:
test_midi = converter.parseFile('./Jazz_midi/Andy.mid')
print('Step 1')
test_notes_to_parse = None

test_parts = instrument.partitionByInstrument(test_midi)
print('Step 2')
if test_parts: 
    # retrieve main instrument part
    notes_to_parse = test_parts.parts[0].recurse()
else: 
    # if the file is not organized by instruments
    notes_to_parse = test_midi.flat.notes

KeyError: 2303681894240

In [4]:
# extract the main notes from all songs and write them to file
def preprocess_midi_files(input_folder, output_file):
    notes = []

    for file in tqdm(glob.glob(os.path.join(input_folder, "*.mid"))):
        try:
            midi = converter.parse(file)
            notes_to_parse = None

            parts = instrument.partitionByInstrument(midi)
            if parts: 
                notes_to_parse = parts.parts[0].recurse()
            else: 
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f'Error loading {file}: {e}')

    with open(output_file, 'w') as f:
        f.write('\n'.join(notes))

input_folder = './Jazz_midi/'
output_file = "jazz.txt"
preprocess_midi_files(input_folder, output_file)

  5%|▍         | 46/934 [01:46<44:12,  2.99s/it]  

Error loading ./Jazz_midi\Andy.mid: 1606126296032


 18%|█▊        | 166/934 [06:00<22:03,  1.72s/it]

Error loading ./Jazz_midi\Chromazone.mid: 1606081379776


 30%|██▉       | 279/934 [10:20<18:06,  1.66s/it]

...

 65%|██████▌   | 609/934 [29:36<27:20,  5.05s/it]  

Error loading ./Jazz_midi\PlayingForTime.mid: 1606203842816


 93%|█████████▎| 864/934 [35:55<02:25,  2.07s/it]

Error loading ./Jazz_midi\WhatsNewInBaltimore.mid: 1605937139472


100%|██████████| 934/934 [37:27<00:00,  2.41s/it]


In [5]:
def prepare_sequences(notes, sequence_length, n_vocab):
    # dictionary used to map notes to numbers 
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # generate training data: sequence of notes => next note
    for i in range(len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    # reshape for (numbers of samples, time steps, features per step)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # scale data
    network_input = network_input / float(n_vocab)
    # one-hot encode
    network_output = to_categorical(network_output)

    return network_input, network_output

with open("/kaggle/input/jazz-processed/jazz.txt", 'r') as f:
    notes = f.read().split('\n')

sequence_length = 50
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, sequence_length, n_vocab)


In [8]:
print(type(network_input[0][0]))
print(network_output[0])
network_input.shape, network_output.shape

<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.

((300743, 50, 1), (300743, 787))

### Define VAE architecture

Inspired from https://keras.io/examples/generative/vae/

In [9]:
class Sampling(Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [11]:
latent_dim = 16

encoder_inputs = Input(shape=(sequence_length, 1))
x = LSTM(128, return_sequences=True)(encoder_inputs)
x = LSTM(128)(x)
z_mean = Dense(latent_dim, name="z_mean")(x)
z_log_var = Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 50, 1)]      0           []                               
                                                                                                  
 lstm_2 (LSTM)                  (None, 50, 128)      66560       ['input_2[0][0]']                
                                                                                                  
 lstm_3 (LSTM)                  (None, 128)          131584      ['lstm_2[0][0]']                 
                                                                                                  
 z_mean (Dense)                 (None, 16)           2064        ['lstm_3[0][0]']                 
                                                                                            

In [12]:
latent_inputs = Input(shape=(latent_dim,))
x = Dense(sequence_length * 128, activation='relu')(latent_inputs)
x = tf.reshape(x, (-1, sequence_length, 128))
x = LSTM(128, return_sequences=True)(x)
x = LSTM(128)(x)
decoder_outputs = Dense(n_vocab, activation='softmax')(x)

decoder = Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 6400)              108800    
                                                                 
 tf.reshape (TFOpLambda)     (None, 50, 128)           0         
                                                                 
 lstm_4 (LSTM)               (None, 50, 128)           131584    
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 787)               101523    
                                                                 
Total params: 473,491
Trainable params: 473,491
Non-trainab

In [13]:
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = Mean(name="total_loss")
        self.reconstruction_loss_tracker = Mean(name="reconstruction_loss")
        self.kl_loss_tracker =  Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        input, output = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(input)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                    tf.keras.losses.categorical_crossentropy(output, reconstruction)
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [24]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=Adam())
vae.fit(network_input, network_output, epochs=5, batch_size=16)

Epoch 1/5
18797/18797 [==============================] - 2978s 158ms/step - loss: 4.8091 - reconstruction_loss: 4.8110 - kl_loss: 4.0413e-07
Epoch 2/5
18797/18797 [==============================] - 2987s 159ms/step - loss: 4.8077 - reconstruction_loss: 4.8061 - kl_loss: 1.5510e-05
Epoch 3/5
18797/18797 [==============================] - 2956s 157ms/step - loss: 4.8077 - reconstruction_loss: 4.8050 - kl_loss: 3.3025e-05
Epoch 4/5
18797/18797 [==============================] - 2897s 154ms/step - loss: 4.8008 - reconstruction_loss: 4.8047 - kl_loss: 2.0464e-05
Epoch 5/5
18797/18797 [==============================] - 2887s 154ms/step - loss: 4.8003 - reconstruction_loss: 4.8046 - kl_loss: 2.7044e-05


In [ ]:
vae.save('VAE')

### Postprocess and save generated music

##### in order to escape the curse of the same argmax prediction, add temperature: less temperature, less chords used by the generator and theoretically closer to the input, more temperature, more chords and more randomized content;

In [90]:
# starting from an initial input sequence, try to predict/reconstruct a new sequence
def generate_music(vae_model, network_input, int_to_note, n_vocab, sequence_length):
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(sequence_length):
        prediction_input = np.reshape(pattern, (1, sequence_length, 1))
        
        z_mean, _, _ = vae_model.encoder.predict(prediction_input)
        prediction = vae_model.decoder.predict(z_mean)
        
        # add temperature to distribution
        prediction = np.log(prediction) / 2 # temperature
        prediction = np.exp(prediction)
        prediction /= np.sum(prediction)
        
        index = np.random.choice(range(n_vocab), p=prediction[0])
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern[1:], [index / float(n_vocab)])
    return prediction_output

int_to_note = dict((number, note) for number, note in enumerate(sorted(set(item for item in notes))))
generated_notes = generate_music(vae, network_input, int_to_note, n_vocab, sequence_length)

1/1 [==============================] - 0s 35ms/step


In [91]:
generated_notes

['F5',
 'D5',
 '11.3.6',
 '1.2.7',
 'E-5',
 '8.9.11.1.4',
 '5.7.10',
 '4.6.9.11',
 '2.4.9',
 '8.1',
 'F3',
 '7.9.11.2',
 '5.8.11.1',
 'B0',
 '0.4.7',
 'E3',
 'F#5',
 'B3',
 '9.11.2.4',
 '7.11.2',
 '5',
 'G#3',
 'A5',
 'E2',
 '2.4.9',
 '2.4.6',
 'F2',
 'F5',
 'E4',
 'C4',
 '11.0.4.7',
 'E3',
 'A3',
 '7.11',
 '7.9.0.3',
 'F5',
 '11.3.4.6',
 '0.6',
 '3.6.8',
 '3.6',
 'A0',
 '2.4.9',
 '8.0.2',
 'B-4',
 '4.6.7',
 '9.11',
 '5.8.10',
 '2.6',
 '11.0.4.7',
 '10.11.2.4']

In [95]:
new_note = note.Note('F#-1') # why is it not recognized?

In [92]:
# creates a midi file from text representation outputted by VAE
def create_midi(prediction_output, output_file):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        try:
            if ('.' in pattern) or pattern.isdigit():
                notes_in_chord = pattern.split('.')
                notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            else:
                new_note = note.Note()
                new_note.pitch = pitch.Pitch(pattern)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)

            offset += 0.5
        except Exception as e:
            print(f'Error loading pattern: {pattern}: {e}')

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

output_file = "VAE_generated_music.mid"
create_midi(generated_notes, output_file)